<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Итоги" data-toc-modified-id="Итоги-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Итоги</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler



In [3]:
df = pd.read_csv('/datasets/insurance.csv')

In [4]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [6]:
df[df.duplicated()].head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
281,1,39.0,48100.0,1,0
488,1,24.0,32900.0,1,0
513,0,31.0,37400.0,2,0
718,1,22.0,32600.0,1,0
785,0,20.0,35800.0,0,0


данные без пропусков, готовы к работе

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** качество линейной регрессии не изменится

**Обоснование:**  
$ a = Xw $  
$ w = (X^T X)^{-1} X^T y $  

$$
a = yw
$$


$ a = XPw $  
$ w_P = ((XP)^T XP)^{-1} (XP)^T y $  
$ w_p = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y $  
$ w_p = P^{-1} (X^T X)^{-1} X^T y $  



$$
a = yw
$$



$$
w_p = P^{-1} w
$$  
$$
a_p = a
$$

## Алгоритм преобразования

**Алгоритм**

генерируем случайные матрицы со стороной, меньшей у исходных данных до тех пор, пока не выйдет обратимая (тк при умножении на необратимую матрицу пострадает качество модели)  
получаем произведение сгенерированной матрицы и исходных данных  
обучаем модель
генерируем предсказание


In [7]:
def generate_inv_matrix(siz):
    state = np.random.RandomState(12345)
    matrix = state.random(size=(siz, siz))
    if np.linalg.det(matrix) != 0:
            return matrix

In [8]:
features = df.drop(['Страховые выплаты'], axis=1)
target = df['Страховые выплаты']

In [9]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)
print(features_train.shape)
print(target_valid.shape)
print('if the latter is 3 times smaller than the former, the division is done correctly')


(3750, 4)
(1250,)
if the latter is 3 times smaller than the former, the division is done correctly


In [10]:
model_orig = LinearRegression()
model_orig.fit(features_train, target_train)
prediction = model_orig.predict(features_valid)
r2 = r2_score(target_valid, prediction)
r2 

0.43522757127026546

In [11]:
multiplication_matrix = generate_inv_matrix(features_train.shape[1])
features_train1 = features_train @ multiplication_matrix
features_valid1 = features_valid @ multiplication_matrix

In [12]:
model_inv = LinearRegression()
model_inv.fit(features_train1, target_train)
prediction = model_inv.predict(features_valid1)
r2 = r2_score(target_valid, prediction)
r2 

0.4352275712706788

## Проверка алгоритма

In [13]:
prediction = model_orig.predict(features_valid)
r2 = r2_score(target_valid, prediction)
r2 

0.43522757127026546

In [14]:
prediction = model_inv.predict(features_valid1)
r2 = r2_score(target_valid, prediction)
r2

0.4352275712706788

In [15]:
features_train

,Пол,Возраст,Зарплата,Члены семьи
3369,1,43.0,36200.0,1
1441,1,34.0,57600.0,0
571,0,32.0,41100.0,1
225,0,36.0,45100.0,1
2558,0,33.0,50600.0,2
...,...,...,...,...
3497,0,42.0,32100.0,0
3492,0,28.0,22700.0,4
2177,1,41.0,44700.0,1
3557,0,22.0,50100.0,4


In [23]:
reverse = features_train1 @ np.linalg.inv(multiplication_matrix)
round(reverse[0] - features_train['Пол'], 5).head(15)

3369   -0.0
1441    0.0
571    -0.0
225     0.0
2558    0.0
4990    0.0
3044    0.0
3703   -0.0
260     0.0
4399   -0.0
4628   -0.0
3939   -0.0
283     0.0
4020   -0.0
3226    0.0
dtype: float64

In [24]:
reverse[0] - features_train['Пол']

3369   -3.849254e-12
1441    1.216227e-11
571    -9.053519e-12
225     5.720367e-12
2558    6.816910e-12
            ...     
3497    1.440139e-12
3492   -4.283398e-12
2177   -1.146949e-11
3557    3.059548e-13
4578    2.187879e-12
Length: 3750, dtype: float64

## Итоги

Данные изначально были чистые, готовые  
Вроде как математически вывел, что умножение исходных данных на коэффициент на качество предсказания Линейной модели не влияет  
Сгенерировал случайную обратимую матрицу, на нее умножил исходные данные, для отмерения результата использовал r2 метрику  
По итогу качество метрики r2 между оригинальными данными и зашифрованными не изменилось
